In [1]:
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from mne.preprocessing import ICA,read_ica

In [2]:
#read the ica file from the folder
sample_data_raw_file=('C:/Users/user/Desktop/ICA/31052019 1_ica.fif')
raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)
raw_temp=raw.copy()#.crop(600., 620.) #Copy the raw data and crop the 30 sec data basically skype call
raw_ch = raw_temp.pick_channels(['F7','F8']) # select only one channel

Opening raw data file C:/Users/user/Desktop/ICA/31052019 1_ica.fif...


<ipython-input-2-060c4e2f3294>:3: RuntimeWarning: This filename (C:/Users/user/Desktop/ICA/31052019 1_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)


    Range : 0 ... 337151 =      0.000 ...  2633.992 secs
Ready.
Reading 0 ... 337151  =      0.000 ...  2633.992 secs...


In [3]:
raw_temp.info['ch_names']

['F7', 'F8']

In [4]:
phas01 =raw_ch.copy().crop(16., 26.)#.pick_channels(['F3']) # copy the data
phas02=raw_ch.copy().crop(180., 190.)
phas31 =raw_ch.copy().crop(1552., 1562.)
phas32=raw_ch.copy().crop(1927., 1937.)
phas41 =raw_ch.copy().crop(2049., 2059.)
phas42=raw_ch.copy().crop(2154., 2164.)
phas51 =raw_ch.copy().crop(2258., 2268.)
phas52=raw_ch.copy().crop(2424., 2434.)


In [5]:
#data0,times0 = ph0[:] #extract times and data
#print(len(data))
#print(data.shape)
#s=1/times
#print(len(s))
#plt.plot(times, data.T)
#plt.show()
#data_sk,times_sk= ph1_sk[:] 
#data_m1,times_m1= ph2_m1[:]
#data_m2,times_m2= ph2_m2[:]
#data_3,times_3= ph_3[:]
#data_4,times_4= ph_4[:]
#data_5,times_5= ph_5[:]

In [6]:
from mne.time_frequency import psd_welch
def eeg_power_band(data):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 4]
        Transformed data.
    """
    # specific frequency bands
    FREQ_BANDS = {#"delta": [0.5, 4.5],
                  #"theta": [4, 8],
                  "alpha": [8, 13]}
                 # "beta": [13, 30],
                  #"gamma": [30, 56]}

    psds, freqs = psd_welch(data, picks=[ 'F7'],fmin=8, fmax=13.)
    
    #print(psds.shape)
    #print(freqs.shape)
    #plt.semilogy(freqs, (psds.T))
    #plt.xlabel('frequency [Hz]')
    #plt.ylabel('Linear spectrum [V RMS]')
    #plt.show()
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:,  (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)    #:,
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

In [7]:
power_01 = eeg_power_band(phas01)
power_02 = eeg_power_band(phas02)
power_31 = eeg_power_band(phas31)
power_32 = eeg_power_band(phas32)
power_41 = eeg_power_band(phas41)
power_42 = eeg_power_band(phas42)
power_51 = eeg_power_band(phas51)
power_52 = eeg_power_band(phas52)
print(power_01)
#print(power_52)

Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
[[0.0972632]]


In [8]:
import pandas as pd
ch_name = ['F7' ]
#column = ["alpha"]
#m0['new'] = 0

#print(F1)


In [9]:
df_0= pd.DataFrame(power_01, columns = ['alpha pha_01'])


df_1= pd.DataFrame(power_02, columns = ['alpha pha_02'])

df_2= pd.DataFrame(power_31, columns = ['alpha pha_31'])

df_3= pd.DataFrame(power_32, columns = ['alpha pha_32'])
df_4= pd.DataFrame(power_41, columns = ['alpha pha_41'])

df_5= pd.DataFrame(power_42, columns = ['alpha pha_42'])
df_6= pd.DataFrame(power_51, columns = ['alpha pha_51'])

df_7= pd.DataFrame(power_52, columns = ['alpha pha_52'])



In [10]:
result =result = pd.concat([df_0,df_1,df_2,df_3,df_4,df_5,df_6,df_7],axis=1, sort=False)
result['sub ID'] = 310520191
result['channel'] = 'F7'
result.set_index('sub ID',inplace=True)
result

,alpha pha_01,alpha pha_02,alpha pha_31,alpha pha_32,alpha pha_41,alpha pha_42,alpha pha_51,alpha pha_52,channel
sub ID,,,,,,,,,
310520191,0.097263,0.093167,0.09531,0.09835,0.095609,0.098206,0.095967,0.095562,F7


In [11]:
#save the data to a csv_file.
result.to_csv(r'C:\Users\user\Desktop\2nd round\F7_01\31052019 1.csv')

In [12]:
def eeg_power_band1(data):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 4]
        Transformed data.
    """
    # specific frequency bands
    FREQ_BANDS = {#"delta": [0.5, 4.5],
                  #"theta": [4, 8],
                  "alpha": [8, 13]}
                 # "beta": [13, 30],
                  #"gamma": [30, 56]}

    psds, freqs = psd_welch(data, picks=[ 'F8'],fmin=8, fmax=13.)
    
    
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:,  (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)    #:,
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

In [13]:
power_001 = eeg_power_band1(phas01)
power_002 = eeg_power_band1(phas02)
power_031 = eeg_power_band1(phas31)
power_032 = eeg_power_band1(phas32)
power_041 = eeg_power_band1(phas41)
power_042 = eeg_power_band1(phas42)
power_051 = eeg_power_band1(phas51)
power_052 = eeg_power_band1(phas52)
print(power_051)
#print(power_52)

Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
[[0.09376495]]


In [14]:
df_00= pd.DataFrame(power_001, columns = ['alpha pha_01'])


df_01= pd.DataFrame(power_002, columns = ['alpha pha_02'])

df_02= pd.DataFrame(power_031, columns = ['alpha pha_31'])

df_03= pd.DataFrame(power_032, columns = ['alpha pha_32'])
df_04= pd.DataFrame(power_041, columns = ['alpha pha_41'])

df_05= pd.DataFrame(power_042, columns = ['alpha pha_42'])
df_06= pd.DataFrame(power_051, columns = ['alpha pha_51'])

df_07= pd.DataFrame(power_052, columns = ['alpha pha_52'])


In [15]:
result =result = pd.concat([df_00,df_01,df_02,df_03,df_04,df_05,df_06,df_07],axis=1, sort=False)
result['sub ID'] = 310520191
result['channel'] = 'F8'
result.set_index('sub ID',inplace=True)
result

,alpha pha_01,alpha pha_02,alpha pha_31,alpha pha_32,alpha pha_41,alpha pha_42,alpha pha_51,alpha pha_52,channel
sub ID,,,,,,,,,
310520191,0.096338,0.092991,0.097798,0.092777,0.091561,0.096052,0.093765,0.097749,F8


In [16]:
#save the data to a csv_file.
result.to_csv(r'C:\Users\user\Desktop\2nd round\F8_01\31052019 1.csv')